# **Trabajo Fin de Master**

### Miguel Angel Ortega Duran 

### Samuel Martínez Camacho

Para mostrar este proyecto elaboramos un chatbot sencillo en base a una serie de patrones y respuestas elaborado en un Json, en el que hemos incluido respuesta a ciertas preguntas sombre los tumores que podemos detectar.

Hemos decidido elaborar con procesamiento del lenguaje natural, para tratar de mejorar el resultado de las  respuestas.

<img src=https://www.talentview.fr/wp-content/uploads/2018/03/DigitasLBi-cree-BotCamp-programme-cocreation-chatbot-marques-T-1040x464-1-1024x457.webp>

Importamos las librerías y herramientas utilizadas en el procesamiento de lenguaje natural y el aprendizaje automático.

* nltk: una librería de procesamiento de lenguaje natural que proporciona herramientas para trabajar con texto en lenguaje natural, como tokenización, etiquetado de partes del discurso, lematización, etc.

* WordNetLemmatizer: una herramienta de nltk para la lematización de palabras, que consiste en reducir una palabra a su forma base o lema.

* json: una librería de Python para trabajar con archivos JSON, que son un formato de datos muy utilizado para el intercambio de información en aplicaciones web.

* pickle: una librería de Python para serializar y deserializar objetos de Python, es decir, para convertir objetos de Python en una secuencia de bytes y viceversa.

* warnings: una librería de Python para el manejo de advertencias y mensajes de advertencia.

* numpy: una librería de Python para trabajar con matrices y arreglos multidimensionales.
tensorflow: una librería de aprendizaje automático y deep learning desarrollada por Google.

* keras: una librería de deep learning que se ejecuta sobre tensorflow.
Sequential: una clase de keras que permite crear modelos de redes neuronales secuenciales, es decir, en los que las capas se apilan una encima de otra en secuencia.

* Dense: una clase de keras que representa una capa densamente conectada de neuronas.

* Activation: una clase de keras que representa una función de activación de neuronas.

* Dropout: una clase de keras que representa una técnica para evitar el sobreajuste en los modelos de redes neuronales.

* SGD: una clase de tensorflow.keras.optimizers que representa un optimizador basado en descenso de gradiente estocástico.

* random: una librería de Python para trabajar con números aleatorios.





In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import random
warnings.filterwarnings('ignore')

Descargamos los paquetes más populares del módulo Natural Language Toolkit (NLTK), que es una biblioteca de Python utilizada para procesar y analizar datos de texto. La descarga de estos paquetes populares es necesaria para utilizar muchas de las funciones de NLTK, como la tokenización y el análisis morfológico. Algunos de los paquetes incluidos en la descarga son stopwords, punkt, wordnet, entre otros.

In [2]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

Cargamos un archivo JSON llamado "intents.json" que contiene datos sobre diferentes intenciones de usuario y sus correspondientes respuestas. Los datos son cargados en formato de diccionario de Python.

La variable "words" se usa para almacenar todas las palabras únicas en los patrones de entrada.

La variable "classes" se usa para almacenar las etiquetas únicas de las intenciones. 

La variable "documents" se usa para almacenar una lista de tuplas, donde cada tupla contiene una lista de palabras para un patrón de entrada y su etiqueta correspondiente.

La lista "ignore_words" contiene signos de puntuación que se ignorarán durante el procesamiento del texto.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/drive/MyDrive/Json/intents.json').read() # lee el fichero de json
intents = json.loads(data_file) # carga el fichero de json

Construimos una lista de palabras (words) que aparecen en las frases de las intenciones. Para hacer esto, el código itera a través de cada intención en el archivo JSON (intents['intents']), luego itera a través de cada patrón de frase en esa intención (intent['patterns']), luego tokeniza cada patrón en palabras individuales usando la función nltk.word_tokenize(). Si una palabra está en la lista de palabras que se deben ignorar (ignore_words), se omite. De lo contrario, la palabra se agrega a la lista words.

Construimos una lista de tuplas (documents) que contienen cada frase de intención junto con su etiqueta de clase correspondiente. Para hacer esto, el código itera a través de cada intención en el archivo JSON (intents['intents']), luego itera a través de cada patrón de frase en esa intención (intent['patterns']), y crea una tupla que contiene una lista de palabras tokenizadas para ese patrón (nltk.word_tokenize(pattern)) y la etiqueta de clase correspondiente (intent['tag']).

Construimos una lista de todas las etiquetas de clase (classes) que se utilizan en el archivo JSON de intenciones. Para hacer esto, el código itera a través de cada intención en el archivo JSON (intents['intents']) y agrega cada etiqueta de clase (intent['tag']) a la lista classes si aún no está presente en la lista (usando la función set() para eliminar duplicados).

In [5]:
words = [word 
         for intent in intents['intents'] 
         for pattern in intent['patterns'] 
         for word in nltk.word_tokenize(pattern) 
         if word not in ignore_words]

documents = [(nltk.word_tokenize(pattern), intent['tag']) 
        for intent in intents['intents'] 
        for pattern in intent['patterns']]

classes = list(set(intent['tag'] 
        for intent in intents['intents']))


Utilizamos la clase SnowballStemmer de la biblioteca NLTK (Natural Language Toolkit) para llevar a cabo la lematización de una palabra en español. 

Descargamos el modelo de datos del WordNet en varios idiomas, incluido el Open Multilingual WordNet (OMW) en su versión 1.4. 

WordNet es una base de datos léxica para el idioma inglés que agrupa las palabras en conjuntos de sinónimos, o "synsets", y proporciona definiciones cortas para cada uno de ellos. 

OMW es una extensión multilingüe de WordNet que cubre varios idiomas, incluido el español. Descargar este recurso con nltk.download('omw-1.4') permite utilizar la herramienta WordNet en español en los proyectos de NLP.

In [6]:
spanish_stemmer = SnowballStemmer('spanish')
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Lematizamos las palabras: utilizando el lematizador NLTK para reducir las palabras a su forma base. Por ejemplo, el lematizador transformaría "corriendo" a "correr".

Ponemos en minúscula cada palabra: convierte todas las palabras a minúsculas para asegurar que las palabras con mayúsculas y minúsculas se traten como iguales.

Eliminamos duplicados: elimina las palabras duplicadas para reducir la cantidad de datos que se procesan.

Ordenamos las clases: ordena las etiquetas de intención de la lista de intenciones en orden alfabético.

Combinamos patrones e intenciones: crea una lista de tuplas, donde cada tupla contiene una lista de tokens de palabras y la etiqueta de intención correspondiente. Cada tupla representa un documento de entrenamiento.

Guardamos los datos procesados: guarda las palabras lematizadas únicas y las etiquetas de intención en archivos de pickle para su uso posterior en el entrenamiento del modelo de chatbot.

In [7]:
# Lematizar, poner en minúscula cada palabra y eliminar duplicados
words = sorted(list(set([lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words])))
# Ordenar las clases
classes = sorted(list(set([intent['tag'] for intent in intents['intents']])))
# Combinación entre patrones e intenciones
documents = [(nltk.word_tokenize(pattern), intent['tag']) for intent in intents['intents'] for pattern in intent['patterns']]
print(len(documents), "documentos\n", documents, "\n")
# Todas las palabras del vocabulario, lematizadas y sin duplicados.
print(len(words), "Palabras lematizadas únicas\n", words, "\n")
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


152 documentos
 [(['Hola'], 'saludo'), (['Buenos', 'días'], 'saludo'), (['Hola', ',', '¿cómo', 'estás', '?'], 'saludo'), (['¡Hola', '!'], 'saludo'), (['¿Qué', 'hace', 'la', 'empresa', '?'], 'informacion'), (['¿En', 'qué', 'se', 'especializa', 'la', 'empresa', '?'], 'informacion'), (['¿Qué', 'tipo', 'de', 'tumores', 'detectan', '?'], 'informacion'), (['¿Cómo', 'funciona', 'la', 'detección', 'mediante', 'CNN', 'de', 'imágenes', 'de', 'resonancia', 'magnética', '?'], 'informacion'), (['¿Qué', 'servicios', 'ofrecen', '?'], 'servicios'), (['¿Cómo', 'puedo', 'programar', 'una', 'cita', '?'], 'servicios'), (['¿Qué', 'pruebas', 'de', 'detección', 'ofrecen', '?'], 'servicios'), (['¿Cuánto', 'cuesta', 'una', 'consulta', '?'], 'servicios'), (['Gracias'], 'agradecimiento'), (['Muchas', 'gracias'], 'agradecimiento'), (['¡Genial', '!'], 'agradecimiento'), (['¡Estoy', 'muy', 'agradecido', '!'], 'agradecimiento'), (['Adiós'], 'despedida'), (['Hasta', 'luego'], 'despedida'), (['¡Nos', 'vemos', 'pronto'

La función create_training_data toma cuatro argumentos: documents, words, classes, y lemmatizer. La función crea datos de entrenamiento para el modelo de chatbot a partir de los documentos, las palabras, las clases y el lematizador proporcionados.

La función comienza inicializando una lista vacía training para almacenar los datos de entrenamiento creados. También crea una lista output_empty de longitud igual a la cantidad de clases. Esta lista se usa para crear la variable de salida de cada documento.

La función itera sobre los documentos proporcionados y procesa cada documento de la siguiente manera:

Crea una lista bag que representa una bolsa de palabras para el documento. La bolsa de palabras es una lista binaria que indica qué palabras del vocabulario están presentes en el documento.

Lematiza cada palabra en el documento y las convierte a minúsculas.

Para cada palabra en el vocabulario, agrega un 1 a la bolsa de palabras si la palabra está presente en el documento y un 0 si no lo está.

Crea una lista output_row que representa la variable de salida del documento. Esta lista es una lista binaria que indica la clase del documento.

Agrega la bolsa de palabras y la variable de salida del documento a la lista de entrenamiento.

Después de crear la lista de entrenamiento, la función la mezcla aleatoriamente, la convierte en un array de NumPy y divide los datos de entrada y salida en dos listas separadas train_x y train_y.

Finalmente, el código llama a la función create_training_data con los argumentos necesarios y muestra un mensaje indicando que los datos de entrenamiento han sido creados.

In [8]:
def create_training_data(documents, words, classes, lemmatizer):
    training = []
    output_empty = [0] * len(classes)

    for doc in documents:
        bag = []
        pattern_words = doc[0]
        pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

        for w in words:
            bag.append(1) if w in pattern_words else bag.append(0)
        
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        
        training.append([bag, output_row])

    random.shuffle(training)
    training = np.array(training)

    train_x = list(training[:,0])
    train_y = list(training[:,1])

    return train_x, train_y

train_x, train_y = create_training_data(documents, words, classes, lemmatizer)
print("Datos de entrenamiento creados")

Datos de entrenamiento creados


Definimos una función llamada build_model que recibe train_x y train_y como argumentos y crea un modelo secuencial de red neuronal con tres capas.

> Bloque con sangría



La primera capa tiene 128 neuronas y utiliza la función de activación ReLU. 

La segunda capa tiene 64 neuronas y también utiliza la función de activación ReLU. 

La tercera y última capa tiene un número de neuronas igual al número de intenciones en train_y, y utiliza la función de activación softmax para predecir la intención de salida. 

El modelo también utiliza la técnica de regularización Dropout para evitar el overfitting.






In [9]:
def build_model():
    # Crear modelo con 3 capas. La primera capa tiene 128 neuronas, la segunda capa tiene 64 neuronas y la ultima capa de salida contiene un número de neuronas
    # igual al número de intenciones para predecir la intención de salida con softmax
    model = Sequential()

    model.add(Dense(128, 
                    input_shape=(len(train_x[0]),), 
                    activation='relu'))
    
    model.add(Dropout(0.5))

    model.add(Dense(64, 
                    activation='relu'))
    
    model.add(Dropout(0.5))
    
    model.add(Dense(len(train_y[0]), 
                    activation='softmax'))
    
    return model

model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               29568     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 38)                2470      
                                                                 
Total params: 40,294
Trainable params: 40,294
Non-trainable params: 0
_________________________________________________________________


Definimos una función Model_fit que toma dos argumentos train_x y train_y que corresponden a los datos de entrenamiento creados previamente. La función entrena un modelo de red neuronal y guarda el modelo entrenado en un archivo "chatbot_model.h5". La función también devuelve el history de entrenamiento del modelo.

El código comienza llamando a la función build_model() para crear una nueva instancia del modelo de red neuronal. A continuación, se compila el modelo con el optimizador Adam y la función de pérdida de entropía cruzada categórica. El modelo se entrena utilizando los datos de entrenamiento proporcionados y se ejecutan 300 épocas con un tamaño de lote de 5.

Después del entrenamiento, el modelo se guarda en un archivo "chatbot_model.h5" utilizando el método save() de Keras. Se muestra un mensaje indicando que el modelo se ha creado y guardado. Finalmente, la función devuelve el historial de entrenamiento del modelo.

En resumen, este código entrena un modelo de red neuronal utilizando los datos de entrenamiento proporcionados y guarda el modelo entrenado en un archivo.

In [12]:
def Model_fit(train_x, train_y):
  model = _model()
  model.compile(loss='categorical_crossentropy', 
                optimizer='adam', 
                metrics=['acc'])
  history = model.fit(np.array(train_x), 
                      np.array(train_y), 
                      epochs=300, 
                      batch_size=5)
  model.save("chatbot_model.h5")
  print("Modelo creado y guardado como chatbot_model.h5")
  return history

history = Model_fit(train_x, train_y)

Epoch 1/300
31/31 [==============================] - 1s 2ms/step - loss: 3.6184 - acc: 0.0263
Epoch 2/300
31/31 [==============================] - 0s 2ms/step - loss: 3.5759 - acc: 0.0789
Epoch 3/300
31/31 [==============================] - 0s 2ms/step - loss: 3.5141 - acc: 0.1184
Epoch 4/300
31/31 [==============================] - 0s 2ms/step - loss: 3.4540 - acc: 0.1316
Epoch 5/300
31/31 [==============================] - 0s 2ms/step - loss: 3.3712 - acc: 0.1250
Epoch 6/300
31/31 [==============================] - 0s 2ms/step - loss: 3.2514 - acc: 0.1908
Epoch 7/300
31/31 [==============================] - 0s 2ms/step - loss: 3.1831 - acc: 0.2237
Epoch 8/300
31/31 [==============================] - 0s 2ms/step - loss: 3.0445 - acc: 0.1908
Epoch 9/300
31/31 [==============================] - 0s 2ms/step - loss: 2.9170 - acc: 0.2763
Epoch 10/300
31/31 [==============================] - 0s 2ms/step - loss: 2.8298 - acc: 0.2566
Epoch 11/300
31/31 [==============================] - 0s 3m

Esta función recibe una oración como entrada y realiza una serie de operaciones para limpiar y procesar esa oración. 

En primer lugar, utiliza la función nltk.word_tokenize de la biblioteca Natural Language Toolkit (nltk) para dividir la oración en una lista de palabras. 

Despues utiliza la función lemmatizer, para encontrar el lema (forma básica) de cada palabra en la lista y las convierte en minúsculas. 

In [ ]:
model = keras.models.load_model('chatbot_model.h5')
print(keras.__version__)

2.11.0


In [ ]:
def clean_up_sentence(sentence):

  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

  return sentence_words

Definimos una función llamada senteces que toma como entrada una oración, una lista de palabras y un argumento opcional show_details que indica si se deben mostrar detalles o no.

La función primero llama a la función clean_up_sentence para tokenizar la oración y reducirla a su forma base usando el lematizador. 

Luego, crea una bolsa de palabras representada por una matriz de tamaño N, donde N es el número de palabras en la lista words. 

Cada elemento en la matriz es inicializado a 0.

La función itera sobre cada palabra en la oración tokenizada y verifica si la palabra se encuentra en la lista de palabras. 

Si la palabra se encuentra en la lista, entonces la posición correspondiente en la matriz de la bolsa de palabras se establece en 1.

Si show_details es verdadero, la función muestra cada palabra encontrada en la bolsa de palabras.

Finalmente, la función devuelve la matriz de la bolsa de palabras.

In [ ]:
def sentences(sentence, words, show_details=True):

  sentence_words = clean_up_sentence(sentence)

  bag = [0]*len(words) 

  for s in sentence_words:  
      for i,w in enumerate(words):
          if w == s: 
              bag[i] = 1
              if show_details:
                  print ("encontrado en la bolsas de palabras: %s" % w)
  return(np.array(bag))

Tomamos una oración como entrada junto con un modelo entrenado y devuelve una lista de intenciones con sus respectivas probabilidades. 

Primero, usamos la función "sentences" para convertir la oración en una matriz de la bolsa de palabras y luego usa el modelo para predecir la probabilidad de cada intención. 

Luego, filtra las predicciones que tienen una probabilidad por debajo de un umbral de error y devuelve una lista ordenada de intenciones y sus probabilidades.

In [ ]:
def predict_class(sentence, model):

  p = sentences(sentence, words,show_details=False)
  res = model.predict(np.array([p]))[0]
  ERROR_THRESHOLD = 0.25

  results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)

  return_list = []

  for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

  return return_list

Definemos la función getResponse que toma una lista de intenciones y un objeto JSON con la definición de esas intenciones. 

La función busca la intención con la probabilidad más alta en la lista y selecciona una respuesta aleatoria de las respuestas asociadas a esa intención.


In [ ]:
def getResponse(ints, intents_json):

  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
      if(i['tag']== tag):
          result = random.choice(i['responses'])
          break
  return result

Función principal del chatbot. 

Toma como entrada una cadena de texto, y utiliza la función predict_class() para predecir la intención detrás del mensaje.

Luego utiliza la función getResponse() para obtener una respuesta aleatoria del archivo JSON que contiene las posibles respuestas según la intención detectada.

Por ultimo, devuelve la respuesta obtenida.

In [ ]:
def chatbot_response(text): 
  ints = predict_class(text, model)
  res = getResponse(ints, intents)
  return res

In [ ]:
start = True

while start:

  query = input('Introduce tu mensaje: ')
  if query in ['quitar','salir','adios']:
      start = False
      continue
  try:
      res = chatbot_response(query)
      print(res)
  except:
      print('Es posible que deba reformular su pregunta.')

Introduce tu mensaje: hola
1/1 [==============================] - 0s 107ms/step
¡Hola! Soy el chatbot de SaveYourLife especializados en la detección de tumores mediante CNN de imágenes de resonancia magnética. ¿En qué puedo ayudarte?
Introduce tu mensaje: que puedes hacer
1/1 [==============================] - 0s 52ms/step
Puedo proporcionar información sobre la detección de tumores cerebrales, responder preguntas relacionadas los tipos de tumores que detectamos y mantener conversaciones diarias. No me consideres un sustituto de un profesional de la medicina. Por favor, busca ayuda si no te sientes satisfecho conmigo.
Introduce tu mensaje: 
1/1 [==============================] - 0s 25ms/step
No estoy seguro de entender.
Introduce tu mensaje: 
1/1 [==============================] - 0s 24ms/step
Por favor, no dudes en hablar conmigo.
Introduce tu mensaje: 
1/1 [==============================] - 0s 24ms/step
No estoy seguro de entender.
Introduce tu mensaje: jajaja
1/1 [==================